In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName("RegresyonTuning") \
.master("local[4]") \
.config("spark.driver.memory","2g") \
.config("spark.executor.memory","4g") \
.getOrCreate()

# veri okuma

In [3]:
# Kaynak: https://www.kaggle.com/kumarajarshi/life-expectancy-who

In [4]:
df = spark.read.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.option("sep",",") \
.load("D:\\Datasets\LifeExpectancyData.csv")

In [5]:
df.toPandas().head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [6]:
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Status: string (nullable = true)
 |-- Life expectancy : double (nullable = true)
 |-- Adult Mortality: integer (nullable = true)
 |-- infant deaths: integer (nullable = true)
 |-- Alcohol: double (nullable = true)
 |-- percentage expenditure: double (nullable = true)
 |-- Hepatitis B: integer (nullable = true)
 |-- Measles : integer (nullable = true)
 |--  BMI : double (nullable = true)
 |-- under-five deaths : integer (nullable = true)
 |-- Polio: integer (nullable = true)
 |-- Total expenditure: double (nullable = true)
 |-- Diphtheria : integer (nullable = true)
 |--  HIV/AIDS: double (nullable = true)
 |-- GDP: double (nullable = true)
 |-- Population: double (nullable = true)
 |--  thinness  1-19 years: double (nullable = true)
 |--  thinness 5-9 years: double (nullable = true)
 |-- Income composition of resources: double (nullable = true)
 |-- Schooling: double (nullable = true)



# Nitelik isimlerini değiştir. Sıralamayı bozmadan.

In [7]:
new_cols = ["Country", "Year", "Status", "label", "AdultMortality",
      "InfantDeaths", "Alcohol", "PercentageExpenditure", "HepatitisB", "Measles", "BMI", "UnderFiveDeaths",
      "Polio", "TotalExpenditure", "Diphtheria", "HIV_AIDS", "GDP", "Population", "Thinness119", "Thinness59",
      "IncomeCompositionOfResources", "Schooling"]

In [8]:
df2 = df.toDF(*new_cols)

In [9]:
df2.toPandas().head()

,Country,Year,Status,label,AdultMortality,InfantDeaths,Alcohol,PercentageExpenditure,HepatitisB,Measles,...,Polio,TotalExpenditure,Diphtheria,HIV_AIDS,GDP,Population,Thinness119,Thinness59,IncomeCompositionOfResources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [10]:
categorical_cols = ["Country","Status"]
'''
numerical_cols = [ "Year", "AdultMortality",
      "InfantDeaths", "Alcohol", "PercentageExpenditure", "HepatitisB", "Measles", "BMI", "UnderFiveDeaths",
      "Polio", "TotalExpenditure", "Diphtheria", "HIV_AIDS", "GDP", "Population", "Thinness119", "Thinness59",
      "IncomeCompositionOfResources", "Schooling"]
'''
numerical_cols = [ "Year", "AdultMortality",
      "InfantDeaths", "Alcohol",  "BMI", "UnderFiveDeaths",
      "TotalExpenditure", "Diphtheria", "HIV_AIDS", "GDP",  
      "IncomeCompositionOfResources", "Schooling"]
label = ["label"]

# Veri Temizliği

In [11]:
df2.describe().toPandas().head()

,summary,Country,Year,Status,label,AdultMortality,InfantDeaths,Alcohol,PercentageExpenditure,HepatitisB,...,Polio,TotalExpenditure,Diphtheria,HIV_AIDS,GDP,Population,Thinness119,Thinness59,IncomeCompositionOfResources,Schooling
0,count,2938,2938,2938,2928,2928,2938,2744,2938,2385,...,2919,2712,2919,2938,2490,2286,2904,2904,2771,2775
1,mean,None,2007.5187202178352,None,69.22493169398912,164.79644808743168,30.303948264125257,4.6028607871720375,738.2512954533823,80.94046121593291,...,82.55018842069202,5.9381895280235915,82.32408359027065,1.7421034717494939,7483.158469138481,1.2753375120052498E7,4.8397038567493205,4.870316804407711,0.6275510645976166,11.992792792792786
2,stddev,None,4.613840940258099,None,9.523867487824305,124.2920790034219,117.92650131339906,4.052412658755658,1987.914858016194,25.070015593018063,...,23.428045949468476,2.498319672155633,23.7169120685726,5.077784531086547,14270.169341515959,6.101209650842794E7,4.420194947144322,4.508882086983007,0.2109035551515931,3.3589197211023554
3,min,Afghanistan,2000,Developed,36.3,1,0,0.01,0.0,1,...,3,0.37,2,0.1,1.68135,34.0,0.1,0.1,0.0,0.0
4,max,Zimbabwe,2015,Developing,89.0,723,1800,17.87,19479.91161,99,...,99,17.6,99,50.6,119172.7418,1.293859294E9,27.7,28.6,0.948,20.7


In [12]:
df3 = df2.na.drop()

In [13]:
df3.describe().toPandas().head()

,summary,Country,Year,Status,label,AdultMortality,InfantDeaths,Alcohol,PercentageExpenditure,HepatitisB,...,Polio,TotalExpenditure,Diphtheria,HIV_AIDS,GDP,Population,Thinness119,Thinness59,IncomeCompositionOfResources,Schooling
0,count,1649,1649,1649,1649,1649,1649,1649,1649,1649,...,1649,1649,1649,1649,1649,1649,1649,1649,1649,1649
1,mean,None,2007.8405093996362,None,69.30230442692543,168.2152819890843,32.55306246209824,4.533195876288676,698.973558049698,79.21770770163735,...,83.56458459672528,5.955924802910845,84.155245603396,1.9838690115221154,5566.031886817592,1.4653625889484541E7,4.850636749545183,4.907762280169795,0.6315512431776829,12.119890842935117
2,stddev,None,4.087710534059055,None,8.796834135238601,125.31041693156172,120.84719049638885,4.029188999208425,1759.2293363477393,25.60466388103202,...,22.450557485191844,2.2993854200363937,21.57919284379662,6.032359689186714,11475.900117274561,7.04603934030564E7,4.599228369708468,4.653756745508691,0.18308872643487184,2.795387538952672
3,min,Afghanistan,2000,Developed,44.0,1,0,0.01,0.0,2,...,3,0.74,2,0.1,1.68135,34.0,0.1,0.1,0.0,4.2
4,max,Zimbabwe,2015,Developing,89.0,723,1600,17.87,18961.3486,99,...,99,14.39,99,50.6,119172.7418,1.293859294E9,27.2,28.2,0.936,20.7


# Veri Ön Hazırlığı

In [14]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.regression import LinearRegression, LinearRegressionModel

# StringIndexer

In [15]:
status_string_indexer = StringIndexer().setInputCol("Status").setOutputCol("StatusIndexed")

# OneHotEncoder

In [16]:
encoder = OneHotEncoderEstimator().setInputCols(["StatusIndexed"]).setOutputCols(["StatusEncoded"])

# VectorAssembler

In [17]:
vector_assembler = VectorAssembler().setInputCols(numerical_cols + encoder.getOutputCols()) \
.setOutputCol("features")

# Lineer Model

In [18]:
linear_regression_object = LinearRegression().setFeaturesCol("features").setLabelCol("label")

# Pipeline

In [19]:
pipeline_object = Pipeline().setStages([status_string_indexer, encoder, vector_assembler, linear_regression_object])

# veri setini ayırma

In [20]:
train_df, test_df = df3.randomSplit([0.8, 0.2], seed=142)

In [21]:
train_df.cache()
test_df.cache()

DataFrame[Country: string, Year: int, Status: string, label: double, AdultMortality: int, InfantDeaths: int, Alcohol: double, PercentageExpenditure: double, HepatitisB: int, Measles: int, BMI: double, UnderFiveDeaths: int, Polio: int, TotalExpenditure: double, Diphtheria: int, HIV_AIDS: double, GDP: double, Population: double, Thinness119: double, Thinness59: double, IncomeCompositionOfResources: double, Schooling: double]

# MODEL TUNING

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# paramGrid

In [24]:
param_grid = ParamGridBuilder() \
.addGrid(linear_regression_object.aggregationDepth, [2,5]) \
.addGrid(linear_regression_object.elasticNetParam, [0.0,0.02]) \
.addGrid(linear_regression_object.epsilon, [1.35, 1.55]) \
.addGrid(linear_regression_object.maxIter, [10, 20]) \
.addGrid(linear_regression_object.regParam, [0.00, 0.01, 0.05]) \
.addGrid(linear_regression_object.solver, ["auto", "normal", "l-bfgs"]) \
.addGrid(linear_regression_object.tol, [1.0E-6, 1.0E-4]) \
.build()

# Cross-Validation

In [26]:
cv = CrossValidator() \
.setEstimator(pipeline_object) \
.setEvaluator(RegressionEvaluator()) \
.setEstimatorParamMaps(param_grid) \
.setNumFolds(5) \
.setParallelism(2)

In [27]:
cv_model = cv.fit(train_df)

In [28]:
cv_model.transform(test_df).select("label","prediction").toPandas().head()

,label,prediction
0,56.2,63.595831
1,57.3,60.953196
2,58.1,61.949129
3,58.6,62.238897
4,72.6,72.930192


# En İyi Model

In [29]:
best_model = cv_model.bestModel

In [30]:
type(best_model)

pyspark.ml.pipeline.PipelineModel

In [31]:
lr_model = best_model.stages[-1]

In [32]:
lr_model.coefficients

DenseVector([-0.1306, -0.0167, 0.0884, -0.0933, 0.0335, -0.0673, 0.1014, 0.0144, -0.4472, 0.0001, 9.6162, 0.9347, -0.7855])

In [33]:
lr_model.intercept

315.44452546889534

In [34]:
lr_model.summary.r2

0.8393963848802052

In [36]:
lr_model.explainParams().split("\n")

['aggregationDepth: suggested depth for treeAggregate (>= 2) (default: 2, current: 2)',
 'elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty (default: 0.0, current: 0.0)',
 'epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. (default: 1.35, current: 1.35)',
 'featuresCol: features column name (default: features, current: features)',
 'fitIntercept: whether to fit an intercept term (default: True)',
 'labelCol: label column name (default: label, current: label)',
 'loss: The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError) (default: squaredError)',
 'maxIter: maximum number of iterations (>= 0) (default: 100, current: 10)',
 'predictionCol: prediction column name (default: prediction)',
 'regParam: regularization parameter (>= 0) (default: 0.0, current: 0.0)',
 'solver: The solver algorithm for optimization. Supported 